# Preparing Data: Merging, Cleaning, and Feature Labeling

This notebook merges the economist list (from Notebook 1) with the biographical data (from Notebook 2). The core focus is on data validation, cleanup, and feature engineering, specifically converting raw Wikidata QIDs stored in lists into readable labels (e.g., converting a QID for 'Female' into the word 'Female'). This process prepares the final dataset for analysis.

***Note on Development**: In some sections, Generative AI was selectively used as a coding assistant, particularly for refining the complex regular expression patterns and structuring the batch API requests for efficiency. This use is explicitly documented with comments in the relevant code cells.*


**Table of Content**

1. [Library Imports](#sec1)
2. [Data Loading and Initial Merging](#sec2)
3. [Data Validation and Cleanup](#sec3)
4. [Extracting QIDs (Regex Use)](#sec4)
5. [Mapping Gender and Getting Labels](#sec5)
6. [Data Export](#sec6)
7. [Further Data Refinement](#sec7)

<a id="sec1"></a>
### Library Imports

In [17]:
import pandas as pd
import re
import requests
import time

<a id="sec2"></a>
### Data Loading and Initial Merging

In [3]:
summ = pd.read_csv("../Data/economists_list_with_summaries.csv")
meta = pd.read_csv("../Data/economists_with_wikidata.csv")

# Keep only economists with QIDs
summ = summ.dropna(subset=["qid"])
meta = meta.dropna(subset=["qid"])

print("Summaries after filtering:", summ.shape)
print("Metadata after filtering:", meta.shape)


Summaries after filtering: (1102, 4)
Metadata after filtering: (1102, 11)


In [4]:
merged = summ.merge(meta, on="qid", how="inner")
print("Merged shape:", merged.shape)

Merged shape: (1102, 14)


In [5]:
merged.to_csv("../Data/economists_master_dataset.csv", index=False)

In [8]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1102 entries, 0 to 1101
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name_x           1102 non-null   object 
 1   article_url_x    1102 non-null   object 
 2   qid              1102 non-null   object 
 3   summary_x        1101 non-null   object 
 4   name_y           1102 non-null   object 
 5   article_url_y    1102 non-null   object 
 6   summary_y        1101 non-null   object 
 7   gender_raw       1101 non-null   object 
 8   birthdate_raw    1072 non-null   object 
 9   citizenship_raw  1014 non-null   object 
 10  occupation_raw   1096 non-null   object 
 11  field_raw        466 non-null    object 
 12  birth_year       1072 non-null   float64
 13  gender_qid       1101 non-null   object 
dtypes: float64(1), object(13)
memory usage: 120.7+ KB


In [9]:
merged.head()

,name_x,article_url_x,qid,summary_x,name_y,article_url_y,summary_y,gender_raw,birthdate_raw,citizenship_raw,occupation_raw,field_raw,birth_year,gender_qid
0,Edith Abbott,https://en.wikipedia.org/wiki/Edith_Abbott,Q272731,"Edith Abbott (September 26, 1876 – July 28, 19...",Edith Abbott,https://en.wikipedia.org/wiki/Edith_Abbott,"Edith Abbott (September 26, 1876 – July 28, 19...","[{'entity-type': 'item', 'numeric-id': 6581072...","[{'time': '+1876-09-26T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1876.0,Q6581072
1,Daron Acemoglu,https://en.wikipedia.org/wiki/Daron_Acemoglu,Q718581,"Kamer Daron Acemoğlu (born September 3, 1967) ...",Daron Acemoglu,https://en.wikipedia.org/wiki/Daron_Acemoglu,"Kamer Daron Acemoğlu (born September 3, 1967) ...","[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1967-09-03T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 43, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1967.0,Q6581097
2,Nicola Acocella,https://en.wikipedia.org/wiki/Nicola_Acocella,Q7001311,Nicola Acocella (born 3 July 1939) is an Itali...,Nicola Acocella,https://en.wikipedia.org/wiki/Nicola_Acocella,Nicola Acocella (born 3 July 1939) is an Itali...,"[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1939-07-03T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 172579,...","[{'entity-type': 'item', 'numeric-id': 188094,...",NaN,1939.0,Q6581097
3,Zoltan Acs,https://en.wikipedia.org/wiki/Zoltan_Acs,Q8073604,Zoltan J. Acs (born 1947) is an American econo...,Zoltan Acs,https://en.wikipedia.org/wiki/Zoltan_Acs,Zoltan J. Acs (born 1947) is an American econo...,"[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1947-03-09T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1947.0,Q6581097
4,Henry Carter Adams,https://en.wikipedia.org/wiki/Henry_Carter_Adams,Q518021,"Henry Carter Adams (December 31, 1851 – August...",Henry Carter Adams,https://en.wikipedia.org/wiki/Henry_Carter_Adams,"Henry Carter Adams (December 31, 1851 – August...","[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1851-12-31T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1851.0,Q6581097


<a id="sec3"></a>
### Data Validation and Cleanup

In [11]:
df = pd.read_csv("../Data/economists_master_dataset.csv")
df = df.drop(columns=["name_y", "article_url_y", "summary_y"])

df = df.rename(columns={
    "name_x": "name",
    "article_url_x": "article_url",
    "summary_x": "summary"
})

<a id="sec4"></a>
### Extracting QIDs (Regex Use)

In [12]:
# Helper to extract all QIDs from a string
def extract_all_qids(text):
    if pd.isna(text):
        return []
    return re.findall(r"Q\d+", text)

# Helper to extract first QID (gender is single)
def extract_first_qid(text):
    qids = extract_all_qids(text)
    return qids[0] if qids else None

In [13]:
# Gender
df["gender"] = df["gender_raw"].apply(extract_first_qid)

# Citizenship (list)
df["citizenship"] = df["citizenship_raw"].apply(extract_all_qids)

# Occupation (list)
df["occupation"] = df["occupation_raw"].apply(extract_all_qids)

# Field of work (list)
df["fields_of_work"] = df["field_raw"].apply(extract_all_qids)

In [ ]:
def extract_birth_year(text):
    if pd.isna(text):
        return None
    match = re.search(r"\+(\d{4})-", text)
    if match:
        return int(match.group(1))
    return None

df["birth_year"] = df["birthdate_raw"].apply(extract_birth_year)

In [ ]:
df = df.drop(columns=[
    "gender_raw",
    "birthdate_raw",
    "citizenship_raw",
    "occupation_raw",
    "field_raw",
    "birth_year",
    "gender_qid"
])

# Drop duplicates & rows with missing QIDs
df = df.drop_duplicates(subset="qid", keep="first")
df = df[df["qid"].notna()]

In [23]:
df.head()

,name,article_url,qid,summary,gender,citizenship,occupation,fields_of_work,birth_year
0,Edith Abbott,https://en.wikipedia.org/wiki/Edith_Abbott,Q272731,"Edith Abbott (September 26, 1876 – July 28, 19...",Q6581072,[Q30],"[Q188094, Q2732142, Q7019111, Q36180, Q1622272...","[Q8134, Q205398, Q12483]",1876.0
1,Daron Acemoglu,https://en.wikipedia.org/wiki/Daron_Acemoglu,Q718581,"Kamer Daron Acemoğlu (born September 3, 1967) ...",Q6581097,"[Q43, Q30]","[Q188094, Q1622272, Q482980]",[Q8134],1967.0
2,Nicola Acocella,https://en.wikipedia.org/wiki/Nicola_Acocella,Q7001311,Nicola Acocella (born 3 July 1939) is an Itali...,Q6581097,"[Q172579, Q38]",[Q188094],[],1939.0
3,Zoltan Acs,https://en.wikipedia.org/wiki/Zoltan_Acs,Q8073604,Zoltan J. Acs (born 1947) is an American econo...,Q6581097,"[Q30, Q28]",[Q188094],[Q8134],1947.0
4,Henry Carter Adams,https://en.wikipedia.org/wiki/Henry_Carter_Adams,Q518021,"Henry Carter Adams (December 31, 1851 – August...",Q6581097,[Q30],"[Q188094, Q1622272, Q36180, Q974144, Q66889825...",[Q8134],1851.0


In [24]:
df.to_csv("../Data/economists_master_dataset.csv", index=False)

<a id="sec5"></a>
### Mapping Gender and Getting Labels

In [25]:
gender_map = {
    "Q6581097": "male",
    "Q6581072": "female",
    "Q1052281": "nonbinary",
    "Q2449503": "transgender female",
    "Q2449504": "transgender male",
    "Q48270": "intersex"
}

df["gender_label"] = df["gender"].map(gender_map)

In [26]:
def get_wikidata_label(qid):
    if pd.isna(qid):
        return None
    
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    try:
        r = requests.get(url, headers={"User-Agent": "CS234-Student-Project"})
        data = r.json()
        return data["entities"][qid]["labels"]["en"]["value"]
    except:
        return None

In [ ]:
# Generative AI was used below to assist with the setup

# flatten citizenship lists → unique QIDs
cit_qids = set(q for lst in df["citizenship"].dropna() for q in lst)

# flatten occupation lists
occ_qids = set(q for lst in df["occupation"].dropna() for q in lst)

# flatten fields_of_work lists
field_qids = set(q for lst in df["fields_of_work"].dropna() for q in lst)

all_qids = list(cit_qids | occ_qids | field_qids)

print("Total QIDs to resolve:", len(all_qids))

qid_to_label = {}

for q in all_qids:
    label = get_wikidata_label(q)
    qid_to_label[q] = label
    time.sleep(0.15)

Total QIDs to resolve: 654


In [ ]:
# Generative AI was used below to assist with the setup and debugging process.

import ast
import numpy as np

# Normalize a single cell so it's always: None or a Python list
def normalize_cell(val):
    if val is None or (isinstance(val, float) and pd.isna(val)):
        return None
    
    # If it's already a Python list
    if isinstance(val, list):
        return val

    # If it's a numpy array
    if isinstance(val, np.ndarray):
        return val.tolist()

    # If it's a string representation of a list: "[Q30, Q38]"
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, list):
                return parsed
            else:
                return None
        except:
            return None

    # Otherwise unknown type
    return None

# Apply normalization first
df["citizenship"] = df["citizenship"].apply(normalize_cell)
df["occupation"] = df["occupation"].apply(normalize_cell)
df["fields_of_work"] = df["fields_of_work"].apply(normalize_cell)

# Now lookup labels safely
def convert_list(lst):
    if lst is None:
        return None
    return [qid_to_label.get(q, None) for q in lst]

df["citizenship_label"] = df["citizenship"].apply(convert_list)
df["occupation_label"] = df["occupation"].apply(convert_list)
df["fields_label"] = df["fields_of_work"].apply(convert_list)


<a id="sec6"></a>
### Data Export

In [32]:
df.to_csv("../Data/economists_cleaned.csv", index=False)

print("Cleaning complete.")
df.head()

Cleaning complete.


,name,article_url,qid,summary,gender,citizenship,occupation,fields_of_work,birth_year,gender_label,citizenship_label,occupation_label,fields_label
0,Edith Abbott,https://en.wikipedia.org/wiki/Edith_Abbott,Q272731,"Edith Abbott (September 26, 1876 – July 28, 19...",Q6581072,[Q30],"[Q188094, Q2732142, Q7019111, Q36180, Q1622272...","[Q8134, Q205398, Q12483]",1876.0,female,[United States],"[economist, statistician, social worker, write...","[economics, social work, statistics]"
1,Daron Acemoglu,https://en.wikipedia.org/wiki/Daron_Acemoglu,Q718581,"Kamer Daron Acemoğlu (born September 3, 1967) ...",Q6581097,"[Q43, Q30]","[Q188094, Q1622272, Q482980]",[Q8134],1967.0,male,"[Turkey, United States]","[economist, university teacher, author]",[economics]
2,Nicola Acocella,https://en.wikipedia.org/wiki/Nicola_Acocella,Q7001311,Nicola Acocella (born 3 July 1939) is an Itali...,Q6581097,"[Q172579, Q38]",[Q188094],[],1939.0,male,"[Kingdom of Italy, Italy]",[economist],[]
3,Zoltan Acs,https://en.wikipedia.org/wiki/Zoltan_Acs,Q8073604,Zoltan J. Acs (born 1947) is an American econo...,Q6581097,"[Q30, Q28]",[Q188094],[Q8134],1947.0,male,"[United States, Hungary]",[economist],[economics]
4,Henry Carter Adams,https://en.wikipedia.org/wiki/Henry_Carter_Adams,Q518021,"Henry Carter Adams (December 31, 1851 – August...",Q6581097,[Q30],"[Q188094, Q1622272, Q36180, Q974144, Q66889825...",[Q8134],1851.0,male,[United States],"[economist, university teacher, writer, educat...",[economics]


<a id="sec7"></a>
### Further Data Refinement

In [ ]:
df = df.drop(columns=[
    "gender",
    "citizenship",
    "occupation",
    "fields_of_work"
])

In [45]:
df = df.rename(columns={
    "gender_label": "gender",
    "citizenship_label": "citizenship",
    "occupation_label": "occupation",
    "fields_label": "fields"
    })

In [49]:
df.to_csv("../Data/economists_cleaned.csv", index=False)